In [9]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interactive, FloatSlider

In [10]:
def gabor(x, y, theta, sigma, f, phi, gamma):
    xp = x * np.cos(theta) + y * np.sin(theta)
    yp = -x * np.sin(theta) + y * np.cos(theta)
    envelope = np.exp(-(xp**2 + gamma**2 * yp**2) / (2 * sigma**2))
    carrier = np.cos(2 * np.pi * f * xp + phi)
    return envelope * carrier

In [41]:
def plot_gabor(theta=0, sigma=1, f=0.5, phi=0, gamma=1):
    x = np.linspace(-10, 10, 500)
    y = np.linspace(-10, 10, 500)
    X, Y = np.meshgrid(x, y)
    G = gabor(X, Y, theta, sigma, f, phi, gamma)

    plt.figure(figsize=(6, 6))
    plt.imshow(G, extent=(-10, 10, -10, 10), origin='lower')
    plt.colorbar()

    # Drawing sigma as a vector
    center_x, center_y = 0, 0 
    end_x = center_x + sigma * np.cos(theta)
    end_y = center_y + sigma * np.sin(theta)

    plt.arrow(center_x, center_y, end_x - center_x, end_y - center_y,
              head_width=0.5, head_length=0.5, fc='red', ec='red', length_includes_head=True)
    plt.text(end_x, end_y, r'$\sigma$', fontsize=12, color='red')

    plt.show()

In [42]:
interactive_plot = interactive(plot_gabor,
                               theta=FloatSlider(min=0, max=np.pi, step=np.pi/16, value=0, description='Theta:'),
                               sigma=FloatSlider(min=0.1, max=10, step=0.1, value=1, description='Sigma:'),
                               f=FloatSlider(min=0.1, max=1, step=0.05, value=0.5, description='Frequency:'),
                               phi=FloatSlider(min=0, max=2*np.pi, step=np.pi/16, value=0, description='Phi:'),
                               gamma=FloatSlider(min=0.1, max=10, step=0.1, value=1, description='Gamma:'))
interactive_plot

interactive(children=(FloatSlider(value=0.0, description='Theta:', max=3.141592653589793, step=0.1963495408493…